In [2]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB

# === 데이터 불러오기 ===
df = pd.read_csv('/content/Training_Set1.csv')  # Colab에 업로드한 파일 경로

# === 부대 정보 추가 ===
locations = [f"부대 {chr(c)}" for c in range(ord('A'), ord('Z') + 1)]
df["location"] = np.random.choice(locations, size=len(df))

# === 프로토콜 원-핫 인코딩 및 이진 매핑 ===
df = pd.get_dummies(df, columns=['protocoltype'], prefix=['protocol'])
df["protocol_icmp"] = df["protocol_icmp"].map({True: 1, False: 0})
df["protocol_tcp"] = df["protocol_tcp"].map({True: 1, False: 0})
df["protocol_udp"] = df["protocol_udp"].map({True: 1, False: 0})

# === 서비스 평균 공격률로 수치화 ===
Mean_of_services = df.groupby(['service'])['attack'].mean().to_dict()
df['service'] = df['service'].map(Mean_of_services)

# === 플래그 값 숫자 변환 ===
flag_convert = {"SF":1,"S0":2,"REJ":3,"RSTO":4,'S1':5, "RSTR":6,'S2':7,'S3':8,"OTH":9,'SH':10}
df['flag'] = df['flag'].map(flag_convert)

# === 타겟과 피처 구분 ===
target = df['attack']
features = df.drop(columns=['attack', 'location'])

# === 데이터 분리 ===
X_train, X_val, y_train, y_val, loc_train, loc_val = train_test_split(
    features, target, df['location'], test_size=0.3, random_state=2
)

# === 정규화 ===
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# === 모델 학습 ===
model = MultinomialNB()
model.fit(X_train_scaled, y_train)

# === 정확도 평가 ===
accuracy = model.score(X_val_scaled, y_val)
print(f"모델 정확도: {accuracy:.4f}")

# === 임의 샘플 예측 테스트 ===
idx = random.randint(0, len(X_val) - 1)
sample_data = X_val.iloc[idx]
scaled_sample = X_val_scaled[idx]
location = loc_val.iloc[idx]
prediction = int(model.predict([scaled_sample])[0])

print("\n=== 테스트 샘플 예측 결과 ===")
print(f"부대 위치: {location}")
print(f"예측 결과: {'공격 (1)' if prediction == 1 else '정상 (0)'}")
print(f"입력 피처:\n{sample_data.to_string()}")


모델 정확도: 0.9970

=== 테스트 샘플 예측 결과 ===
부대 위치: 부대 S
예측 결과: 정상 (0)
입력 피처:
duration                    0.000000
service                     0.193143
flag                        1.000000
srcbytes                   11.000000
dstbytes                    0.000000
land                        0.000000
wrongfragment               0.000000
urgent                      0.000000
hot                         0.000000
numfailedlogins             0.000000
loggedin                    0.000000
numcompromised              0.000000
rootshell                   0.000000
suattempted                 0.000000
numroot                     0.000000
numfilecreations            0.000000
numshells                   0.000000
numaccessfiles              0.000000
numoutboundcmds             0.000000
ishostlogin                 0.000000
isguestlogin                0.000000
count                       1.000000
srvcount                    1.000000
serrorrate                  0.000000
srvserrorrate               0.000000
rerro